# Laboratorium 2
## Stanisław Dudiak

### Setup

In [92]:
# Python imports
import psycopg2 as pg
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Connect to database
connection = pg.connect(host = 'pgsql-196447.vipserv.org', port = 5432, user = 'wbauer_adb',
    password = 'adb2020', dbname = 'wbauer_adb')


### 1.

In [93]:
# Read the lengyh of 'category' table
df = pd.read_sql('select * from category',connection)
print ("Number of film categories:", df.shape[0])

Number of film categories: 16


### 2.

In [94]:
# Order movie categories by name
df = pd.read_sql('SELECT name FROM category ORDER BY name',connection)
display('Sorted Movie Categories:', df)


'Sorted Movie Categories:'

,name
0,Action
1,Animation
2,Children
3,Classics
4,Comedy
5,Documentary
6,Drama
7,Family
8,Foreign
9,Games


### 3.

In [95]:
# Order movies by release date ascending and descending, then read the first position i each dataframe
df = pd.read_sql('SELECT title FROM film ORDER BY release_year DESC',connection)
print('Newest movie:', df['title'].iloc[0])
df = pd.read_sql('SELECT title FROM film ORDER BY release_year ASC',connection)
print('Oldest movie:', df['title'].iloc[0])
# Every movie is from the same year, so oldest and newest are the same movie

Newest movie: Chamber Italian
Oldest movie: Chamber Italian


### 4.

In [96]:
# Read the length of dataframe created from movies rented between selected dates
from datetime import datetime
df = pd.read_sql(('SELECT rental_date FROM rental WHERE rental_date BETWEEN %(min_date)s AND %(max_date)s'),connection,
params={"min_date":datetime(2005,7,1,0,0), "max_date":datetime(2005,8,1,0,0)})
print("Number of rentals in selected period of time: ", df.shape[0])


Number of rentals in selected period of time:  6709


### 5.

In [97]:
# Read the length of dataframe created from movies rented between selected dates
df = pd.read_sql(('SELECT rental_date FROM rental WHERE rental_date BETWEEN %(min_date)s AND %(max_date)s'),connection,
params={"min_date":datetime(2010,1,1,0,0), "max_date":datetime(2011,2,1,0,0)})
print("Number of rentals in selected period of time: ", df.shape[0])


Number of rentals in selected period of time:  0


### 6.

In [98]:
# List payments in ascendig order by amount 
df = pd.read_sql('SELECT amount FROM payment ORDER BY payment ASC',connection)
print("Largest payment is:", df['amount'].iloc[0])


Largest payment is: 7.99


### 7.

In [99]:
# Get id's of selected countries
country_ids = pd.read_sql('SELECT country_id,country FROM country WHERE country IN %(countries)s',connection,
params={"countries":("Poland", "Nigeria", "Bangladesh")})['country_id']

# Match coutry -> city id's
city_ids = pd.read_sql('SELECT city, city_id FROM city WHERE country_id IN %(ids)s',connection,
params={"ids":tuple(country_ids.values.tolist())})['city_id']

# Match city -> address id's
address_ids = pd.read_sql('SELECT address, address_id FROM address WHERE city_id IN %(ids)s',connection,
params={"ids":tuple(city_ids.values.tolist())})['address_id']

# Find custmer's names by address id's
customers = pd.read_sql('SELECT first_name, last_name FROM customer WHERE address_id IN %(ids)s',connection,
params={"ids":tuple(address_ids.values.tolist())})

print("Customers from selected countries:")
display(customers)

Customers from selected countries:


,first_name,last_name
0,Carol,Garcia
1,Michelle,Clark
2,Marilyn,Ross
3,Gladys,Hamilton
4,Marjorie,Tucker
5,Bertha,Ferguson
6,Elsie,Kelley
7,Constance,Reid
8,Jo,Fowler
9,Leah,Curtis


### 8.

In [100]:
# Get staff's address id's
staff = pd.read_sql('SELECT first_name, last_name, address_id FROM staff' ,connection)
address_ids = tuple(staff['address_id'].values.tolist())

# Find addresses with id's
staff_addressess = pd.read_sql('SELECT *  FROM address WHERE address_id IN %(ids)s' ,connection,
params={"ids":address_ids}) 

# Merge dataframes by address id
staff_with_adresses = staff.merge(staff_addressess)
display(staff_with_adresses)

,first_name,last_name,address_id,address,address2,district,city_id,postal_code,phone,last_update
0,Mike,Hillyer,3,23 Workhaven Lane,None,Alberta,300,,14033335568,2006-02-15 09:45:30
1,Jon,Stephens,4,1411 Lillydale Drive,None,QLD,576,,6172235589,2006-02-15 09:45:30


### 9.

In [101]:
# Match country id with staff address id
country_ids = pd.read_sql('SELECT city, country_id FROM city WHERE city_id IN %(ids)s',connection,
params={"ids":tuple(staff_addressess['city_id'].values.tolist())})

countries = pd.read_sql('SELECT country FROM country WHERE country_id IN %(ids)s AND country IN %(countries)s',connection,
params={"ids":tuple(country_ids['country_id'].values.tolist()), "countries":("Argentina", "Spain")})
print(countries)
# No workers are from Argentine or Spain

Empty DataFrame
Columns: [country]
Index: []


### 10.

In [102]:
# Match category_name -> category_id -> film_id -> inventory_id and find id's in rental
db_string = "SELECT name FROM category WHERE category_id IN \
    (SELECT category_id FROM film_category WHERE film_id IN \
        (SELECT film_id FROM inventory WHERE inventory_id IN \
            (SELECT inventory_id FROM rental)))"
categories = pd.read_sql(db_string, connection)
print(categories)
# Movies from all categories have been rented

           name
0        Action
1     Animation
2      Children
3      Classics
4        Comedy
5   Documentary
6         Drama
7        Family
8       Foreign
9         Games
10       Horror
11        Music
12          New
13       Sci-Fi
14       Sports
15       Travel


### 11.

In [103]:
# Same as before, only longer
db_string = "SELECT name FROM category WHERE category_id IN \
    (SELECT category_id FROM film_category WHERE film_id IN \
        (SELECT film_id FROM inventory WHERE inventory_id IN \
            (SELECT inventory_id FROM rental WHERE customer_id IN \
                (SELECT customer_id FROM customer WHERE address_id IN \
                    (SELECT address_id FROM address WHERE city_id IN \
                        (SELECT city_id FROM city WHERE country_id IN \
                            (SELECT country_id FROM country WHERE name = 'America')))))))"
categories = pd.read_sql(db_string, connection)
print(categories)

# No movies from america have been rented

Empty DataFrame
Columns: [name]
Index: []


### 12.

In [104]:
# Same as before, only longer
db_string = "SELECT title FROM film WHERE film_id IN \
    (SELECT film_id FROM film_actor WHERE actor_id IN \
        (SELECT actor_id FROM actor WHERE first_name IN %(names)s AND last_name IN %(surnames)s))"
titles = pd.read_sql(db_string, connection,
params={"names":("Olympia", "Julia", "Ellen"), "surnames":("Pfeiffer", "Zellweger", "Presley")})
print("Movies with selected actors")
print(titles)


Movies with selected actors
                title
0      Express Lonely
1     Bilko Anonymous
2    Effect Gladiator
3   Contact Anonymous
4         Others Soup
..                ...
64      Hanky October
65    Tourist Pelican
66   Detective Vision
67       None Spiking
68     Homeward Cider

[69 rows x 1 columns]
